# FloPy

### Demo of netCDF and shapefile export capabilities within the flopy export module. 

In [1]:
from __future__ import print_function
import os
import sys

# run installed version of flopy or add local path
try:
    import flopy
except:
    fpth = os.path.abspath(os.path.join('..', '..'))
    sys.path.append(fpth)
    import flopy

print(sys.version)
print('flopy version: {}'.format(flopy.__version__))

flopy is installed in /Users/jdhughes/Documents/Development/flopy_git/flopy_fork/flopy
3.7.3 | packaged by conda-forge | (default, Jul  1 2019, 14:38:56) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
flopy version: 3.2.13


Load our old friend...the Freyberg model

In [2]:
nam_file = "freyberg.nam"
model_ws = os.path.join("..", "data", "freyberg_multilayer_transient")
ml = flopy.modflow.Modflow.load(nam_file, model_ws=model_ws, check=False)

We can see the ``Modelgrid`` instance has generic entries, as does ``start_datetime``

In [3]:
ml.modelgrid

xll:622241.1904510253; yll:3343617.741737109; rotation:15.0; proj4_str:+proj=utm +zone=14 +ellps=WGS84 +datum=WGS84 +units=m +no_defs; units:meters; lenuni:2

In [4]:
ml.modeltime.start_datetime

'1/1/2015'

Setting the attributes of the ``ml.modelgrid`` is easy:

In [5]:
proj4_str = "+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs"
ml.modelgrid.set_coord_info(xoff=123456.7, yoff=765432.1, angrot=15.0, proj4=proj4_str)
ml.dis.start_datetime = '7/4/1776'

In [6]:
ml.modeltime.start_datetime

'7/4/1776'

### Some netCDF export capabilities:

#### Export the whole model (inputs and outputs)

In [7]:
# make directory
pth = os.path.join('data', 'netCDF_export')
if not os.path.exists(pth):
    os.makedirs(pth)

In [8]:
fnc = ml.export(os.path.join(pth, ml.name+'.in.nc'))
hds = flopy.utils.HeadFile(os.path.join(model_ws,"freyberg.hds"))
flopy.export.utils.output_helper(os.path.join(pth, ml.name+'.out.nc'), ml, {"hds":hds})

#### export a single array to netcdf or shapefile

In [9]:
# export a 2d array
ml.dis.top.export(os.path.join(pth, 'top.nc'))
ml.dis.top.export(os.path.join(pth, 'top.shp'))

wrote data/netCDF_export/top.shp


#### sparse export of stress period data for a boundary condition package  
* excludes cells that aren't in the package (aren't in `package.stress_period_data`)  
* by default, stress periods with duplicate parameter values (e.g., stage, conductance, etc.) are omitted
(`squeeze=True`); only stress periods with different values are exported  
* argue `squeeze=False` to export all stress periods

In [10]:
ml.drn.stress_period_data.export(os.path.join(pth, 'drn.shp'), sparse=True)

wrote data/netCDF_export/drn.shp


#### Export a 3d array

In [11]:
#export a 3d array
ml.upw.hk.export(os.path.join(pth, 'hk.nc'))
ml.upw.hk.export(os.path.join(pth, 'hk.shp'))

wrote data/netCDF_export/hk.shp


#### Export a number of things to the same netCDF file

In [12]:
# export lots of things to the same nc file
fnc = ml.dis.botm.export(os.path.join(pth, 'test.nc'))
ml.upw.hk.export(fnc)
ml.dis.top.export(fnc)

# export transient 2d
ml.rch.rech.export(fnc)


### Export whole packages to a netCDF file

In [13]:
# export mflist
fnc = ml.wel.export(os.path.join(pth, 'packages.nc'))
ml.upw.export(fnc)
fnc.nc

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.6, ACDD-1.3, flopy 3.2.13
    date_created: 2019-11-15T20:47:00Z
    geospatial_vertical_positive: up
    geospatial_vertical_min: -25.0
    geospatial_vertical_max: 4.832500457763672
    geospatial_vertical_resolution: variable
    featureType: Grid
    namefile: freyberg.nam
    model_ws: ../data/freyberg_multilayer_transient
    exe_name: mf2005.exe
    modflow_version: mfnwt
    create_hostname: IGSAAAHMLT40179
    create_platform: Darwin
    create_directory: /Users/jdhughes/Documents/Development/flopy_git/flopy_fork/examples/Notebooks
    solver_head_tolerance: -999
    solver_flux_tolerance: -999
    flopy_sr_xll: 123456.7
    flopy_sr_yll: 765432.1
    flopy_sr_rotation: 15.0
    flopy_sr_proj4_str: +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs
    start_datetime: 7/4/1776
    dimensions(sizes): time(1097), layer(3), y(40), x(20)
    variables(dimensions): int32 crs(),

### Export the whole model to a netCDF

In [14]:
fnc = ml.export(os.path.join(pth, 'model.nc'))
fnc.nc

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.6, ACDD-1.3, flopy 3.2.13
    date_created: 2019-11-15T20:47:00Z
    geospatial_vertical_positive: up
    geospatial_vertical_min: -25.0
    geospatial_vertical_max: 4.832500457763672
    geospatial_vertical_resolution: variable
    featureType: Grid
    namefile: freyberg.nam
    model_ws: ../data/freyberg_multilayer_transient
    exe_name: mf2005.exe
    modflow_version: mfnwt
    create_hostname: IGSAAAHMLT40179
    create_platform: Darwin
    create_directory: /Users/jdhughes/Documents/Development/flopy_git/flopy_fork/examples/Notebooks
    solver_head_tolerance: -999
    solver_flux_tolerance: -999
    flopy_sr_xll: 123456.7
    flopy_sr_yll: 765432.1
    flopy_sr_rotation: 15.0
    flopy_sr_proj4_str: +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs
    start_datetime: 7/4/1776
    dimensions(sizes): time(1097), layer(3), y(40), x(20)
    variables(dimensions): int32 crs(),